# Fine-tuning on the reference dataset
In this notebook, we'll demonstrate fine-tuning a pre-trained CNN on the 30-isolate classification task shown in Figure 2. In this example, fine-tuning serves to update the CNN to new measurement parameters. This code illustrates the procedure described in the `CNN architecture & training details` section in the Methods. Note that for speed and clarity, this demo only trains on a single randomly selected train and validation split.

## Loading data
The first step is to load the fine-tuning dataset.

In [1]:
from time import time
t00 = time()
import numpy as np

In [2]:
X_fn = './data/X_finetune.npy'
y_fn = './data/y_finetune.npy'
X = np.load(X_fn)
y = np.load(y_fn)
print(X.shape, y.shape)

(3000, 1000) (3000,)


## Loading pre-trained CNN
Now we set up a ResNet CNN and load weights that we previously trained for the 30-isolate task using the full training dataset. 

In [3]:
from resnet import ResNet
import os
import torch

In [4]:
# CNN parameters
layers = 6
hidden_size = 100
block_size = 2
hidden_sizes = [hidden_size] * layers
num_blocks = [block_size] * layers
input_dim = 1000
in_channels = 64
n_classes = 30
os.environ['CUDA_VISIBLE_DEVICES'] = '{}'.format(0)
cuda = torch.cuda.is_available()

In [5]:
# Load trained weights for demo
cnn = ResNet(hidden_sizes, num_blocks, input_dim=input_dim,
                in_channels=in_channels, n_classes=n_classes)
if cuda: cnn.cuda()
cnn.load_state_dict(torch.load(
    './pretrained_model.ckpt', map_location=lambda storage, loc: storage))

## Fine-tuning
Now we can fine-tune the pre-trained CNN on the new fine-tuning dataset. In the experiments reported in the paper, we fine-tune across 5 randomly selected train and validation splits, but here we show just one split for clarity. We also only train for one epoch here in the interest of time. To train the CNN to convergence, we recommend setting the number of epochs to ~30.

In [6]:
from datasets import spectral_dataloader
from training import run_epoch
from torch import optim

### Train/val split
We split the fine-tuning dataset into train and validation sets. We randomly sample 10% of the dataset to use as a validation set.

In [7]:
p_val = 0.1
n_val = int(3000 * p_val)
idx_tr = list(range(3000))
np.random.shuffle(idx_tr)
idx_val = idx_tr[:n_val]
idx_tr = idx_tr[n_val:]

In [8]:
# Fine-tune CNN
epochs = 1 # Change this number to ~30 for full training
batch_size = 10
t0 = time()
# Set up Adam optimizer
optimizer = optim.Adam(cnn.parameters(), lr=1e-3, betas=(0.5, 0.999))
# Set up dataloaders
dl_tr = spectral_dataloader(X, y, idxs=idx_tr,
    batch_size=batch_size, shuffle=True)
dl_val = spectral_dataloader(X, y, idxs=idx_val,
    batch_size=batch_size, shuffle=False)
# Fine-tune CNN for first fold
best_val = 0
no_improvement = 0
max_no_improvement = 5
print('Starting fine-tuning!')
for epoch in range(epochs):
    print(' Epoch {}: {:0.2f}s'.format(epoch+1, time()-t0))
    # Train
    acc_tr, loss_tr = run_epoch(epoch, cnn, dl_tr, cuda,
        training=True, optimizer=optimizer)
    print('  Train acc: {:0.2f}'.format(acc_tr))
    # Val
    acc_val, loss_val = run_epoch(epoch, cnn, dl_val, cuda,
        training=False, optimizer=optimizer)
    print('  Val acc  : {:0.2f}'.format(acc_val))
    # Check performance for early stopping
    if acc_val > best_val or epoch == 0:
        best_val = acc_val
        no_improvement = 0
    else:
        no_improvement += 1
    if no_improvement >= max_no_improvement:
        print('Finished after {} epochs!'.format(epoch+1))
        break

print('\n This demo was completed in: {:0.2f}s'.format(time()-t00))

Starting fine-tuning!
 Epoch 1: 0.00s
  Train acc: 78.52
  Val acc  : 84.33

 This demo was completed in: 185.40s


The accuracies seen here are not representative of the accuracies achieved when training on the full dataset until convergence. To do this, increase the number of epoches. This code demonstrates how a pre-trained CNN can be fine-tuned and evaluated using randomly selected train/validation splits.